<a href="https://colab.research.google.com/github/alex-smith-uwec/AI_Spring2025/blob/main/MDP_gamma_curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

def value_iteration(gamma, epsilon=0.001):
    """
    Performs value iteration on the grid world.

    Grid layout (0-indexed):
      Columns: 0,1,2,3
      Rows: 0,1,2   (row 0 is bottom)

    Special cells:
      - Terminal states:
            (3,2) with reward +1   (top-right)
            (3,1) with reward -1   (middle-right)
      - Blocked cell (inaccessible):
            (1,1)
      - All other states get a reward of -0.04 per move.

    Actions:
      - 'up', 'down', 'left', 'right'
      - When an action is chosen, the agent moves in the intended direction with probability 0.8,
        and in each of the two perpendicular directions with probability 0.1.
      - If a move would hit the wall, go off the grid, or hit the blocked cell, the agent remains in place.

    Returns:
      A tuple (U, terminals) where U is a dictionary mapping each accessible state (tuple (x,y))
      to its computed utility and terminals is the dictionary of terminal states.
    """
    # Grid dimensions
    width, height = 4, 3

    # Define the blocked cell and terminal states
    wall = (1, 1)  # blocked cell (inaccessible)
    terminals = {(3, 2): 1, (3, 1): -1}

    # Build list of states (all grid cells except the wall)
    states = [(x, y) for x in range(width) for y in range(height) if (x, y) != wall]

    # Set rewards: terminal states have their fixed rewards; all others get -0.04.
    rewards = {s: terminals[s] if s in terminals else -0.04 for s in states}

    # Define possible actions and their corresponding movement vectors.
    actions = ['up', 'down', 'left', 'right']
    action_vectors = {
        'up':    (0, 1),
        'down':  (0, -1),
        'left':  (-1, 0),
        'right': (1, 0)
    }
    # Perpendicular moves for each action.
    perpendicular = {
        'up':    ['left', 'right'],
        'down':  ['left', 'right'],
        'left':  ['up', 'down'],
        'right': ['up', 'down']
    }

    def move(state, vector):
        """Returns the resulting state when moving from 'state' using 'vector'.
           If the move goes off the grid or into the wall, the state remains unchanged."""
        next_state = (state[0] + vector[0], state[1] + vector[1])
        if (next_state[0] < 0 or next_state[0] >= width or
            next_state[1] < 0 or next_state[1] >= height or
            next_state == wall):
            return state
        return next_state

    # Initialize utilities: nonterminal states start at 0; terminal states fixed.
    U = {s: 0 for s in states}
    for s in terminals:
        U[s] = terminals[s]

    # Special-case: if gamma == 0, one iteration is enough because future rewards are ignored.
    if gamma == 0:
        for s in states:
            if s not in terminals:
                U[s] = rewards[s]
        print("Gamma = 0, converged in one iteration.")
        return U, terminals

    iteration = 0
    while True:
        U_new = U.copy()
        delta = 0
        for s in states:
            if s in terminals:
                continue  # terminal states remain fixed
            action_values = []
            for a in actions:
                intended_state = move(s, action_vectors[a])
                perp1_state = move(s, action_vectors[perpendicular[a][0]])
                perp2_state = move(s, action_vectors[perpendicular[a][1]])
                expected_util = (0.8 * U[intended_state] +
                                 0.1 * U[perp1_state] +
                                 0.1 * U[perp2_state])
                action_values.append(expected_util)
            best_action_value = max(action_values)
            U_new[s] = rewards[s] + gamma * best_action_value
            delta = max(delta, abs(U_new[s] - U[s]))
        U = U_new
        iteration += 1
        # Convergence criterion adjusted for gamma
        if delta < epsilon * (1 - gamma) / gamma:
            break
    print(f"Gamma = {gamma} converged after {iteration} iterations.")
    return U, terminals



In [6]:
def print_grid(U):
    """
    Prints the utility values in a grid format.
    Uses the same grid conventions as in value_iteration.
    """
    width, height = 4, 3
    wall = (1, 1)
    grid = []
    for y in reversed(range(height)):  # top row first
        row = []
        for x in range(width):
            s = (x, y)
            if s == wall:
                row.append(" WALL ")
            elif s in U:
                row.append(f"{U[s]:6.3f}")
            else:
                row.append("  N/A ")
        grid.append(row)
    for row in grid:
        print("\t".join(row))
print_grid(U)

 0.647	 0.753	 0.855	 1.000
 0.557	 WALL 	 0.569	-1.000
 0.465	 0.386	 0.451	 0.230


In [8]:

def vectorize_utilities(U, terminals, width=4, height=3, wall=(1,1)):
    """
    Extracts the utilities of the 9 nonterminal states into a vector.

    The vector is built in row-major order (from bottom row upward, left to right),
    skipping the wall and terminal states.

    Returns:
      A list of 9 utility values.
    """
    vector = []
    for y in range(height):
        for x in range(width):
            s = (x, y)
            if s == wall or s in terminals:
                continue
            vector.append(U[s])
    return vector



In [9]:
# Loop over gamma values from 0 to 0.95 in steps of 0.05,
# compute the utility vector for each, and store into a matrix.
gammas = np.arange(0, 1.0, 0.05)  # will include 0.0 up to 0.95
matrix = []

for g in gammas:
    U, terminals = value_iteration(g, epsilon=0.001)
    vec = vectorize_utilities(U, terminals)
    matrix.append(vec)

matrix = np.array(matrix)
print("\nUtility matrix (each row corresponds to a gamma value):")
# print(matrix)


Gamma = 0, converged in one iteration.
Gamma = 0.05 converged after 2 iterations.
Gamma = 0.1 converged after 2 iterations.
Gamma = 0.15000000000000002 converged after 3 iterations.
Gamma = 0.2 converged after 3 iterations.
Gamma = 0.25 converged after 4 iterations.
Gamma = 0.30000000000000004 converged after 4 iterations.
Gamma = 0.35000000000000003 converged after 5 iterations.
Gamma = 0.4 converged after 6 iterations.
Gamma = 0.45 converged after 7 iterations.
Gamma = 0.5 converged after 7 iterations.
Gamma = 0.55 converged after 8 iterations.
Gamma = 0.6000000000000001 converged after 9 iterations.
Gamma = 0.65 converged after 10 iterations.
Gamma = 0.7000000000000001 converged after 11 iterations.
Gamma = 0.75 converged after 12 iterations.
Gamma = 0.8 converged after 13 iterations.
Gamma = 0.8500000000000001 converged after 14 iterations.
Gamma = 0.9 converged after 15 iterations.
Gamma = 0.9500000000000001 converged after 20 iterations.

Utility matrix (each row corresponds to a